0 - create dataset  
1 - run experiment  
2 - plot counterfactuals  
3 - plot embeddings  
4 - plot results  

In [1]:
import shutil
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager

from src.utils.context import Context

from utils_martina.my_utils import *

In [2]:
# Set patient and record
patient_id = "chb03"
record_id = "02"

shutil.copyfile(f"dataset_{patient_id}_{record_id}.json", "data/datasets/gcs/grafo_corr_sliding.json")

'data/datasets/gcs/grafo_corr_sliding.json'

In [3]:
logs_path = "..\\..\\explainability\GRETEL-repo\\output\\logs\\"
eval_manager_path = "..\\..\\explainability\GRETEL-repo\\output\\eval_manager\\"

## Removing cache dataset

In [4]:
remove_cache_dataset = True

if remove_cache_dataset:
    folder = ".\data\cache\datasets"

    for file in os.listdir(folder):
        if file.startswith("GCS-"):  # Checks if the file begins with "GCS-"
            cache_path = os.path.join(folder, file)
            if os.path.isfile(cache_path):
                os.remove(cache_path)
                print(f"Removed: {cache_path}")

## Run Gretel experiment

In [5]:
# NOTA: Se si blocca, andare a vedere se ci sono dei file OracleTorch provenienti dal futuro

In [6]:
path = 'config/GCS-GCN.jsonc'
# path = 'config/GCS-SVM.jsonc'
# path = 'config/GCS-KNN.jsonc'

print(f"Generating context for: {path}")
context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info(
    "Creating the evaluation manager......................................................."
)

if 'doe-triplets' in context.conf:
    context.logger.info("Creating the TRIPLET evaluators........................................................")
    eval_manager = TripletsEvaluatorManager(context)
if 'do-pairs' in context.conf:
    context.logger.info("Creating the PAIRED evaluators...............................................................") ### Usiamo questo
    eval_manager = PairedEvaluatorManager(context)
else:
    context.logger.info("Creating the evaluators...............................................................")
    eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)

eval_manager.evaluate()

# Additional stuff
context.logger.info(f"graph_ids: {eval_manager._evaluators[0].get_instance_and_counterfactual_ids()}")
context.logger.info(f"graph_metrics: {eval_manager._evaluators[0].get_instance_and_counterfactual_graph_metrics()}")

Generating context for: config/GCS-GCN.jsonc
2025-03-03 15:10:03,1549248222 | INFO | 20744 - Executing: config/GCS-GCN.jsonc Run: -1
2025-03-03 15:10:03,1549248241 | INFO | 20744 - Creating the evaluation manager.......................................................
2025-03-03 15:10:03,1549248258 | INFO | 20744 - Creating the PAIRED evaluators...............................................................
2025-03-03 15:10:03,1549248406 | INFO | 20744 - Creating: GCS-189d68770f99189694c24d6a27f87039
2025-03-03 15:10:03,1549248406 | INFO | 20744 - Instantiating: src.dataset.generators.gcs.GCS
2025-03-03 15:10:07,1549252603 | INFO | 20744 - Saved: GCS-189d68770f99189694c24d6a27f87039
2025-03-03 15:10:07,1549252679 | INFO | 20744 - Created: GCS-189d68770f99189694c24d6a27f87039
2025-03-03 15:10:07,1549252727 | INFO | 20744 - Instantiating: src.oracle.nn.gcn.DownstreamGCN
2025-03-03 15:10:07,1549252786 | INFO | 20744 - Instantiating: torch.optim.RMSprop
2025-03-03 15:10:07,1549252786 | INFO

In [7]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)

## _______________________________________________________________

In [8]:
import datetime

now = datetime.datetime.now()
print("Ultima esecuzione completa:", now.strftime("%d/%m/%Y, ore %H:%M"))

Ultima esecuzione completa: 03/03/2025, ore 16:15
